# Part 1

## Task 3

### Data loading

We use several methods to preprocess the data.

1. ```transforms.RandomHorizontalFlip()```: This randomly flips each image horizontally to augment the dataset by creating variations of the images.
2. ```transforms.RandomCrop(32, padding=4)```: This crops the image to a 32x32 size with a padding of 4 pixels, which adds randomness to the dataset by simulating slight shifts.
3. ```transforms.ToTensor()```: Convert to tensor.
4. ```transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))```: This normalizes the image tensor using the mean (0.4914, 0.4822, 0.4465) and standard deviation (0.2470, 0.2435, 0.2616) for each RGB channel, which are the computed means and standard deviations of the CIFAR-10 dataset. Parameters referred https://stackoverflow.com/questions/69747119/pytorch-cifar10-images-are-not-normalized

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from MLPnet import MLPnet

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### The model, the criterion and the optimizer

The model, see ```MLPnet.py```, added two units into MLP:

1. ```BatchNormalization```: reduces internal covariate shift.
2. ```Dropout```: randomly sets the output of some neurons in the neural network to zero. This operation makes the model independent of specific neurons or feature combinations during training.

The ```weight_decay``` is used to avoid overfitting.

In [2]:
model = MLPnet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

### Train and test

In [3]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


def test(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.4f}')


train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)
test(model, test_loader)

Epoch [1/20], Loss: 1.9003, Validation Accuracy: 0.3791
Epoch [2/20], Loss: 1.7231, Validation Accuracy: 0.4162
Epoch [3/20], Loss: 1.6469, Validation Accuracy: 0.4378
Epoch [4/20], Loss: 1.6076, Validation Accuracy: 0.4413
Epoch [5/20], Loss: 1.5758, Validation Accuracy: 0.4534
Epoch [6/20], Loss: 1.5494, Validation Accuracy: 0.4565
Epoch [7/20], Loss: 1.5284, Validation Accuracy: 0.4670
Epoch [8/20], Loss: 1.5102, Validation Accuracy: 0.4782
Epoch [9/20], Loss: 1.4980, Validation Accuracy: 0.4752
Epoch [10/20], Loss: 1.4825, Validation Accuracy: 0.4851
Epoch [11/20], Loss: 1.4679, Validation Accuracy: 0.4846
Epoch [12/20], Loss: 1.4543, Validation Accuracy: 0.4915
Epoch [13/20], Loss: 1.4465, Validation Accuracy: 0.5022
Epoch [14/20], Loss: 1.4359, Validation Accuracy: 0.4965
Epoch [15/20], Loss: 1.4258, Validation Accuracy: 0.5037
Epoch [16/20], Loss: 1.4164, Validation Accuracy: 0.4967
Epoch [17/20], Loss: 1.4040, Validation Accuracy: 0.5221
Epoch [18/20], Loss: 1.4059, Validation 